## Exercise 5
In this exercise are we going to apply basic numerical linear algebra methods.  
More specifically, we are going to solve a linear system of equations with a tridiagonal NxN matrix.

#### 1. Gaussian elimination
At first we implement a method to use the Gaussian elimination on a matrix. This is important since it gives us a triangular matrix in row-echelon form.  

If we have a problem of the form Ax=b with A as a matrix, x and b vectors and we're looking for x, we have to apply the same steps of calculation on b as we do on A.  
To do this more efficiently we trianglify a new matrix, which is simply the original matrix A with the vector b added as a new column.

In [1]:
import numpy as np

In [2]:
#subroutine using the gauss algorithm to create a triagonal matrix from an entered Matrix A
def trianglify(A):
    
    N=len(A)
    Amat=A.astype("float")#makes sure there is no unnecessary rounding  
    i=0
    j=0
    #iterates through rows of Matrix
    while i < N:
        #Step 1: puts one at diagonal spot in matrix
        Amat[i]=Amat[i]*(1/Amat[i][i]) 
        #Step 2: subtract multiple of this row from all rows below to create zeros
        j=i
        while j < N:
            if i==j:
                j=j+1
            else:
                Amat[j]=Amat[j]-Amat[j][i]*Amat[i]
                j=j+1
        i+=1
        j=0 
    return Amat#returns triagonal matrix




#### 2. Backwards substitution
The next step to solve a linear system of equation is the backwards substitution.  

In [ ]:
#Function for backsubstituting in the gauss agorithm
#assumes matrix Amat contains the solution of the set of equation in the rightest column
def backsub(Amat):
    N=len(Amat)-1
    x=np.ones(N+1)#will contain the solved x-vector
    nb=N+1#column index containing the solution vector b
    
    #starting with the lower right corner of the triangular matrix, starts back substitution of already found x's
    i=N
    j=N
    while i>=0:
        j=N
        x[i]=Amat[i][nb]
        while j>i:
            x[i]=x[i]-Amat[i][j]*x[j]
            j-=1       
        i-=1
        
    return x


#### 3. Tridiagonal matrix and solution
After implementing the different steps to solve a linear system of equations we now want to put it together into one routine.  
  
We are looking at the special case of a NxN tridiagonal matrix.  
Before we can solve the problem, we have to create the matrix, given the values of $\vec{a}$, $\vec{b}$, $\vec{c}$ and $\vec{y}$. 

Then we put all the implemented functions to use in a single solving function that creates a Matrix of the specified tridiagonal form, and solves it using the gauss algorithm for triangularisation and then the backsubstituion.

In [3]:
#function creates a tridiagonal Matrix of the shape given in homework
def create_tridiag(a,b,c):
    N=len(b)
    n=N-1
    A=np.zeros((N,N))
      
    j=0
    while j<=n:
        A[j][j]=b[j]
        if j==0:
            A[j][j+1]=c[j+1]
        if j ==n:
            A[j][j-1]=a[n-1]
        if (j<n)and (j>0 ):
            A[j][j-1]=a[j-1]
            A[j][j+1]=c[j-1]
        j+=1
    return A

#given the arrays containing the parameters for the as bs and cs, function calls upon create_tridiag(),
#trianglify() and backsub()to solve the set of linear equations 
def tridiag_solve(a,b,c,y):
    A = create_tridiag(a,b,c)
    N=len(b)
    n=N-1
    
    x=np.zeros(len(b))#vector for solution x
    
    yarr=np.array([y])#asures proper shape of y vector

    Ab=np.concatenate((A,yarr.T),axis=1)#join created Matrix with solution y vector
    x=backsub(trianglify(Ab))#solve joined MAtrix
    return A,Ab,x #tridiagonal matrix A, the result for the triangularmatrix Ab and the found solution x




#### 4. Test the method
In this case we set N to 10, all of the a and c values to -1, the b values to 3 and y,the solution values, to 0.2.  
We will look if the method works and check the result later.

In [7]:
#set parameters of tridiag. matrix as specified
a=np.ones(9)*-1
b=np.ones(10)*3
c=np.ones(9)*-1

y=np.ones(10)*0.2

#calls upon functions to create the necessary matrixes and solve it.
A,Ab,x=tridiag_solve(a,b,c,y)
print("tridiagonal Matrix with concatenated solution vector y (according to specs in homework)")
print(Ab)
print("\n solved  x:")
print(x)

tridiagonal Matrix with concatenated solution vector y (according to specs in homework)
[[ 3.  -1.   0.   0.   0.   0.   0.   0.   0.   0.   0.2]
 [-1.   3.  -1.   0.   0.   0.   0.   0.   0.   0.   0.2]
 [ 0.  -1.   3.  -1.   0.   0.   0.   0.   0.   0.   0.2]
 [ 0.   0.  -1.   3.  -1.   0.   0.   0.   0.   0.   0.2]
 [ 0.   0.   0.  -1.   3.  -1.   0.   0.   0.   0.   0.2]
 [ 0.   0.   0.   0.  -1.   3.  -1.   0.   0.   0.   0.2]
 [ 0.   0.   0.   0.   0.  -1.   3.  -1.   0.   0.   0.2]
 [ 0.   0.   0.   0.   0.   0.  -1.   3.  -1.   0.   0.2]
 [ 0.   0.   0.   0.   0.   0.   0.  -1.   3.  -1.   0.2]
 [ 0.   0.   0.   0.   0.   0.   0.   0.  -1.   3.   0.2]]

 solved  x:
[0.12359551 0.17078652 0.18876404 0.19550562 0.19775281 0.19775281
 0.19550562 0.18876404 0.17078652 0.12359551]


Now to test if everything works, we simply multiply the tridiagonal matrix A from our equation A*x=y and substitute in our solution for x, to see if the result is the same as teh y we started out with.

In [8]:
#test if functions work, by multiplying found x with the created Tridiagonal Matrix A 
Am=np.matrix(A)
xm=np.matrix(x).T
ytest=Am*xm
print("our result for y using tridiagonal A as specified and our solution x:")
print(ytest)
 

our result for y using tridiagonal A as specified and our solution x:
[[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]


As one can see clearly, the result matches y perfectly A*x=y !

Seems like our functions work properly.